In [1]:
import numpy as np
import pandas as pd
import scipy.io as sio

In [4]:
source_mat_file = './mpii_human_pose_v1_u12_1.mat'

In [5]:
mat = sio.loadmat(source_mat_file, struct_as_record = False)
rel = mat['RELEASE']
print(rel, type(rel), rel.shape)

[[<scipy.io.matlab.mio5_params.mat_struct object at 0x000001346D7DEE20>]] <class 'numpy.ndarray'> (1, 1)


In [6]:
obj_rel = rel[0,0]
print(obj_rel._fieldnames)

['annolist', 'img_train', 'version', 'single_person', 'act', 'video_list']


In [7]:
annolist = obj_rel.__dict__['annolist']
img_tra = obj_rel.__dict__['img_train']
act = obj_rel.__dict__['act']
print(annolist.shape,img_tra.shape,act.shape)

(1, 24987) (1, 24987) (24987, 1)


In [8]:
n = annolist.shape[1]
data_arr = ['NAME','r ankle_X','r ankle_Y', 'r knee_X','r knee_Y', 'r hip_X','r hip_Y', 'l hip_X','l hip_Y', 'l knee_X','l knee_Y', 'l ankle_X','l ankle_Y','pelvis_X','pelvis_Y','thorax_X','thorax_Y','upper neck_X','upper neck_Y', 'head top_X','head top_Y', 'r wrist_X','r wrist_Y','r elbow_X','r elbow_Y', 'r shoulder_X','r shoulder_Y','l shoulder_X','l shoulder_Y','l elbow_X','l elbow_Y', 'l wrist_X','l wrist_Y','Scale','Activity','Category']
print(data_arr)

['NAME', 'r ankle_X', 'r ankle_Y', 'r knee_X', 'r knee_Y', 'r hip_X', 'r hip_Y', 'l hip_X', 'l hip_Y', 'l knee_X', 'l knee_Y', 'l ankle_X', 'l ankle_Y', 'pelvis_X', 'pelvis_Y', 'thorax_X', 'thorax_Y', 'upper neck_X', 'upper neck_Y', 'head top_X', 'head top_Y', 'r wrist_X', 'r wrist_Y', 'r elbow_X', 'r elbow_Y', 'r shoulder_X', 'r shoulder_Y', 'l shoulder_X', 'l shoulder_Y', 'l elbow_X', 'l elbow_Y', 'l wrist_X', 'l wrist_Y', 'Scale', 'Activity', 'Category']


In [9]:
data = pd.DataFrame(columns=data_arr)

In [12]:
for ix in range(0,annolist.shape[1]):
    if img_tra[0,ix] == 0:
        continue
    temp_arr = []
    obj_list = annolist[0,ix]
    obj_act = act[ix,0]
    
    rect =obj_list.__dict__['annorect']
    img_d = obj_list.__dict__['image']
    if rect.shape[0] ==0:
        continue
        
    obj_rect = rect[0,0]
    obj_img = img_d[0,0]
    
    
    if 'annopoints' not in obj_rect._fieldnames:
        continue
    
    name_d = obj_img.__dict__['name']
    name = name_d[0]
    temp_arr.append(name)
    annopoints = obj_rect.__dict__['annopoints']
    if annopoints.shape[0]==0:
        continue
    obj_points = annopoints[0,0]
    points = obj_points.__dict__['point']
    cnt = 0
    px = 0
    
    for n in range(0,32):
        temp_arr.append(-1)
    
    
    for px in range(0,points.shape[1]):
        po = points[0,px]
        po_id = po.__dict__['id']
        
        po_x = po.__dict__['x']
        po_y = po.__dict__['y']
        ind = 2*po_id[0][0]+1
        temp_arr[ind] = po_x[0][0]
        temp_arr[ind+1] = po_y[0][0]
       
        
        
    scale = obj_rect.__dict__['scale']
    temp_arr.append(scale[0][0])
    
    activity = act[ix,0]
    
    a_n = activity.act_name
    c_n = activity.cat_name
    
    if a_n.shape[0]==0:
        temp_arr.append(a_n)
    else:
        temp_arr.append(activity.act_name[0])
    if c_n.shape[0]==0:
        temp_arr.append(c_n)
    else:
        temp_arr.append(activity.cat_name[0])
    
    temp_data_f = pd.DataFrame([temp_arr],columns=data_arr)
    
    data = pd.concat([data,temp_data_f]) 
    
    if ix%1000==0:
        print(ix)

2000
4000
6000
9000
10000
11000
12000
13000
14000
15000
16000
18000
21000
22000
23000
24000


In [13]:
data.head(10)

,NAME,r ankle_X,r ankle_Y,r knee_X,r knee_Y,r hip_X,r hip_Y,l hip_X,l hip_Y,l knee_X,...,r shoulder_Y,l shoulder_X,l shoulder_Y,l elbow_X,l elbow_Y,l wrist_X,l wrist_Y,Scale,Activity,Category
0,015601864.jpg,620,394,616,269,573,-71,647,-68,661,...,-89,692,-71,693,-16,688,313,3.021046,curling,sports
0,015599452.jpg,-1,-1,-1,-1,806,543,720,593,-1,...,281,719,299,711,516,545,466,5.641276,curling,sports
0,005808361.jpg,804,711,816,510,908,438,1040,454,906,...,-3,1067,-3,1167,353,1142,478,4.718488,curling,sports
0,086617615.jpg,301,461,305,375,-55,340,294,342,335,...,261,314,264,327,320,362,346,2.681349,curling,sports
0,060111501.jpg,980,322,896,318,865,-8,943,-30,948,...,-109,923,123,995,-93,961,-33,3.806403,curling,sports
0,070807258.jpg,461,398,509,335,517,-38,570,-53,-1,...,280,652,265,636,356,621,417,2.400120,curling,sports
0,002058449.jpg,918,456,659,518,713,413,979,288,1222,...,270,877,-9,1112,384,1012,489,8.280870,curling,sports
0,021233911.jpg,-1,-1,355,408,269,354,432,333,464,...,-88,389,-109,492,259,413,321,3.781047,sitting quietly,inactivity quiet/light
0,018182497.jpg,325,349,341,327,326,302,385,277,409,...,-25,360,-69,406,-38,374,258,3.523671,sitting quietly,inactivity quiet/light
0,018340451.jpg,392,390,388,300,289,296,302,283,395,...,-62,292,-69,320,262,362,276,1.949178,"sitting, talking in person, on the phone, comp...",miscellaneous


In [14]:
data.to_csv('mpii_dataset.csv')